In [3]:
import ee
import geemap

In [4]:

ee.Initialize()
print(ee.String('Hello World from EE!').getInfo())



Hello World from EE!


In [5]:
# 태안해안해상국립공원 영역을 추출하는 함수 정의
def get_boundary():
    roi = ee.FeatureCollection("WCMC/WDPA/current/polygons") \
        .filter(ee.Filter.eq("NAME", "Taeanhaean")) \
        .geometry()
    return roi

# 변산반도국립공원 경계 추출
region = get_boundary()

In [7]:
# Earth Engine 초기화
ee.Initialize()

Map = geemap.Map()

roi = Map.user_roi
if roi is None:
    roi = ee.FeatureCollection("WCMC/WDPA/current/polygons") \
        .filter(ee.Filter.eq("NAME", "Taeanhaean")) \
        .geometry().bounds()

Map.addLayer(roi)
Map.centerObject(roi, 12)

# !pip install ffmpeg-python -q

timelapse = geemap.sentinel1_timelapse(
    roi,
    out_gif=None,
    start_year=2014,
    end_year=2022,
    start_date='01-01',
    end_date='12-31',
    bands=['VV'],
    frequency='day',
    reducer='median',
    date_format=None,
    palette='Greys',
    vis_params=None,
    dimensions=768,
    frames_per_second=1,
    crs='EPSG:3857',
    overlay_data=None,
    overlay_color='black',
    overlay_width=1,
    overlay_opacity=1.0,
    title=None,
    title_xy=('2%', '90%'),
    add_text=True,
    text_xy=('2%', '2%'),
    text_sequence=None,
    font_type='arial.ttf',
    font_size=20,
    font_color='white',
    add_progress_bar=True,
    progress_bar_color='blue',
    progress_bar_height=5,
    add_colorbar=False,
    colorbar_width=6.0,
    colorbar_height=0.4,
    colorbar_label=None,
    colorbar_label_size=12,
    colorbar_label_weight='normal',
    colorbar_tick_size=10,
    colorbar_bg_color=None,
    colorbar_orientation='horizontal',
    colorbar_dpi='figure',
    colorbar_xy=None,
    colorbar_size=(300, 300),
    loop=0, mp4=False,
    fading=False,
    orbitProperties_pass='DESCENDING')

# 생성된 타임랩스 보여주기
geemap.show_image(timelapse)



Generating URL...
Total request size (91433472 pixels) must be less than or equal to 26214400 pixels.
The input gif file does not exist.


Output()

## STRM 데이터 다운

In [5]:
# SRTM Digital Elevation 30m
SRTM = ee.Image('USGS/SRTMGL1_003')
elevation = SRTM.select('elevation')
visParams = {
    'min': 0,
    'max': 5000,
    'palette': ['000000','006600', '009900','33CC00','996600','CC9900','CC9966','FFFFFF']}

# 지도 생성
Map = geemap.Map()

Map.centerObject(roi, 4) # 경계 중심으로 지도 확대
Map.addLayer(elevation, visParams, 'Elevation')
Map

Map(center=[36.63914727726935, 126.21386541654387], controls=(WidgetControl(options=['position', 'transparent_…

In [6]:
# 태안해안 영역으로 잘라내기
elevation_roi = elevation.clipToCollection(roi)
elevation_roi
# # 데이터를 Google Drive에 저장
# geemap.ee_export_image_to_drive(
#     elevation_kr,
#     description='elevation',
#     folder='export',
#     region=kr.geometry(),
#     scale=100
# )

## 토지 피복도 : Dynamic World

In [10]:
# 날짜 범위 설정
start_date = '2022-01-01'
end_date = '2023-01-01'

# Sentinel-2 영상 합성(image composite)
image = geemap.dynamic_world_s2(region, start_date, end_date)
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
Map.addLayer(image, vis_params, 'Sentinel-2 image')

# 지도 출력
Map

Map(bottom=25894.0, center=[36.62434536776987, 126.91680908203126], controls=(WidgetControl(options=['position…

In [11]:
# Dynamic World 토지피복 합성(land cover composite)
landcover = geemap.dynamic_world(region, start_date, end_date, return_type='hillshade')
Map.addLayer(landcover, {}, 'Land Cover')
Map

Map(bottom=51583.0, center=[36.41465185677698, 125.79208374023439], controls=(WidgetControl(options=['position…

In [31]:
import ee
import geemap
import pandas as pd

# Google Earth Engine 초기화
ee.Initialize()

# Sentinel-1 이미지의 후방산란 차이 계산
def get_backscatter_difference(image):
    return image.select('VV').subtract(image.select('VH')).rename('VV_VH')

# 주어진 조건에 따라 Sentinel-1 이미지 컬렉션 반환
def fetch_sentinel1_images(region, start_date, end_date):
    return (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
            .filterBounds(region)
            .filterDate(start_date, end_date))

# 주어진 이미지를 Sentinel-1 이미지의 해상도와 투영에 맞게 재투영
def harmonize_to_sentinel(image, reference_image):
    return image.reproject(reference_image.projection(), scale=10)

# Sentinel-1과 다른 데이터 소스 결합
def combine_datasets(image):
    dynamic_world = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').select(['discrete_classification'])
    srtm = ee.Image("USGS/SRTMGL1_003").select(['elevation'])
    
#     harmonized_dynamic_world = harmonize_to_sentinel(dynamic_world, image)
#     harmonized_srtm = harmonize_to_sentinel(srtm, image)
    
    return image.addBands([
        get_backscatter_difference(image),
        dynamic_world,
        srtm
    ])

# 이미지 컬렉션을 pandas DataFrame으로 변환
def convert_to_dataframe(sampled_data):
    df = geemap.ee_to_pandas(sampled_data, ['VV_VH', 'discrete_classification', 'elevation', 'longitude', 'latitude', 'system:index'])
    df['image_id'] = df['system:index'].apply(lambda x: x.split('_')[0])
    df['date'] = pd.to_datetime(df['image_id'].str.extract(r'(\d{8})')[0], format='%Y%m%d')
    return df[['image_id', 'date', 'VV_VH', 'discrete_classification', 'elevation', 'longitude', 'latitude']]

# 메인 함수
def main(region_name, start_date, end_date):
    roi = ee.FeatureCollection("WCMC/WDPA/current/polygons").filter(ee.Filter.eq("NAME", region_name)).geometry().bounds()
    
    sentinel1_images = fetch_sentinel1_images(roi, start_date, end_date)
    combined_data = sentinel1_images.map(combine_datasets)
    
    # 5000개 임의 지점에서 다중-밴드 예측 이미지 간 공변량 값 추출
    DataCor = predictors.sample(scale=GrainSize, numPixels=5000, geometries=True) # 5000개 임의 지점 생성
    PixelVals = predictors.sampleRegions(collection=DataCor, scale=GrainSize, tileScale=16) # 공변량 값 추출
    
    df = convert_to_dataframe(sampled_data)
    
    print(df.head())
    df.to_csv('combined_data.csv', index=False)

# 실행
main("Taeanhaean", '2020-12-01', '2020-12-31')

NameError: name 'predictors' is not defined

In [ ]:
# NASA SRTM Digital Elevation 30m: 디지털 고도 데이터
Terrain = ee.Algorithms.Terrain(ee.Image("USGS/SRTMGL1_003"))

In [41]:
import ee
import geemap
import pandas as pd

# Initialize Earth Engine
ee.Initialize()

# ROI 설정
def get_roi(region_name):
    return ee.FeatureCollection("WCMC/WDPA/current/polygons").filter(ee.Filter.eq("NAME", region_name)).geometry().bounds()

# Sentinel-1 데이터 로드
def load_sentinel1_data(roi, start_date, end_date):
    return (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(roi)
            .filterDate(ee.Date(start_date), ee.Date(end_date))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .select(['VV', 'VH']))

# SRTM 데이터 로드 및 경사도 계산
def load_srtm_data(roi):
    srtm_elevation = ee.Image('USGS/SRTMGL1_003')
    slope = ee.Terrain.slope(srtm_elevation)
    return srtm_elevation.addBands(slope).select(['elevation', 'slope'])

def load_dynamic_world_data(roi, start_date, end_date):
    return (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
            .filterDate(ee.Date(start_date), ee.Date(end_date))
            .first())

# 합치는 함수
def combine_datasets(image, dynamic_world):
    srtm = load_srtm_data(image.geometry())
    combined_image = image.addBands(srtm).addBands(dynamic_world.select(['label', 'water', 'trees', 'grass']))  # Add more bands if needed
    return combined_image.set('system:index', image.get('system:index'))

# 메인 함수
def main(region_name, start_date, end_date):
    roi = get_roi(region_name)
    sentinel1_images = load_sentinel1_data(roi, start_date, end_date)
    dynamic_world = load_dynamic_world_data(roi, start_date, end_date)
    combined_data = sentinel1_images.map(lambda image: combine_datasets(image, dynamic_world))

    # 1000m 픽셀단위로 샘플링
    sampled_data = combined_data.map(lambda image: image.sample(roi, scale=1000)).flatten()

    # 데이터프레임으로 변환
    df = geemap.ee_to_pandas(sampled_data)
    df['sentinel-1_filename'] = df['system:index']
    df['date'] = pd.to_datetime(df['sentinel-1_filename'].str.extract(r'(\d{8})')[0], format='%Y%m%d')
    df = df[['sentinel-1_filename', 'date', 'VV', 'VH', 'elevation', 'slope', 'longitude', 'latitude']]
    return df


# 실행 예제
region_name = "Taeanhaean"
start_date = '2020-12-01'
end_date = '2020-12-31'
df = main(region_name, start_date, end_date)
print(df.head())



Exception: Element.propertyNames: Parameter 'element' is required.

In [2]:
import pandas as pd

a = pd.read_csv("/Users/o_jungii/Downloads/sentinel1_data_202201.csv")
a

,system:index,Date,FileName,VH,VV,VV_VH,latitude,longitude,.geo
0,S1A_IW_GRDH_1SDV_20220105T093153_20220105T0932...,_1SDV_20,S1A_IW_GRDH_1SDV_20220105T093153_20220105T0932...,"[-26.700623053432857, -26.258569643683845, -26...","[-20.04663767732432, -19.650310703023074, -19....",[],"[36.375929957493845, 36.375929957493845, 36.37...","[126.33232419393862, 126.33322250922274, 126.3...","{""type"":""MultiPoint"",""coordinates"":[]}"
1,S1A_IW_GRDH_1SDV_20220117T093152_20220117T0932...,_1SDV_20,S1A_IW_GRDH_1SDV_20220117T093152_20220117T0932...,"[-26.602799314874545, -25.633166611470614, -25...","[-15.445707429926745, -14.85401211204418, -15....",[0.9867788],"[36.375929957493845, 36.375929957493845, 36.37...","[126.33232419393862, 126.33322250922274, 126.3...","{""type"":""MultiPoint"",""coordinates"":[]}"
2,S1A_IW_GRDH_1SDV_20220129T093152_20220129T0932...,_1SDV_20,S1A_IW_GRDH_1SDV_20220129T093152_20220129T0932...,"[-27.18217400304919, -25.842634601894563, -26....","[-18.079308532567314, -19.315020610940252, -19...",[0.9561364],"[36.375929957493845, 36.375929957493845, 36.37...","[126.33232419393862, 126.33322250922274, 126.3...","{""type"":""MultiPoint"",""coordinates"":[]}"
3,S1A_IW_GRDH_1SDV_20220210T093152_20220210T0932...,_1SDV_20,S1A_IW_GRDH_1SDV_20220210T093152_20220210T0932...,"[-27.00214240270697, -27.210963918428096, -25....","[-18.441372029393396, -19.96445165839703, -21....",[0.9430799],"[36.375929957493845, 36.375929957493845, 36.37...","[126.33232419393862, 126.33322250922274, 126.3...","{""type"":""MultiPoint"",""coordinates"":[]}"
4,S1A_IW_GRDH_1SDV_20220306T093151_20220306T0932...,_1SDV_20,S1A_IW_GRDH_1SDV_20220306T093151_20220306T0932...,"[-26.770789577761466, -26.538178830928846, -26...","[-17.72989045605255, -16.605119610305618, -18....",[],"[36.375929957493845, 36.375929957493845, 36.37...","[126.33232419393862, 126.33322250922274, 126.3...","{""type"":""MultiPoint"",""coordinates"":[]}"
5,S1A_IW_GRDH_1SDV_20220318T093151_20220318T0932...,_1SDV_20,S1A_IW_GRDH_1SDV_20220318T093151_20220318T0932...,"[-25.99833860378116, -26.15876580954751, -26.7...","[-18.549407143817444, -19.812927464007043, -19...",[],"[36.375929957493845, 36.375929957493845, 36.37...","[126.33232419393862, 126.33322250922274, 126.3...","{""type"":""MultiPoint"",""coordinates"":[]}"
6,S1A_IW_GRDH_1SDV_20220330T093152_20220330T0932...,_1SDV_20,S1A_IW_GRDH_1SDV_20220330T093152_20220330T0932...,"[-26.897025408259854, -26.21491004539715, -26....","[-17.941841913657864, -18.71348617149781, -19....",[],"[36.375929957493845, 36.375929957493845, 36.37...","[126.33232419393862, 126.33322250922274, 126.3...","{""type"":""MultiPoint"",""coordinates"":[]}"
7,S1A_IW_GRDH_1SDV_20220411T093152_20220411T0932...,_1SDV_20,S1A_IW_GRDH_1SDV_20220411T093152_20220411T0932...,"[-27.07112401233313, -26.21317703664772, -26.7...","[-25.43572327545387, -25.4000617031085, -25.68...",[],"[36.375929957493845, 36.375929957493845, 36.37...","[126.33232419393862, 126.33322250922274, 126.3...","{""type"":""MultiPoint"",""coordinates"":[]}"
8,S1A_IW_GRDH_1SDV_20220423T093153_20220423T0932...,_1SDV_20,S1A_IW_GRDH_1SDV_20220423T093153_20220423T0932...,"[-26.521044262713616, -25.88748258243667, -25....","[-26.55690125054095, -25.944278386307865, -26....",[],"[36.375929957493845, 36.375929957493845, 36.37...","[126.33232419393862, 126.33322250922274, 126.3...","{""type"":""MultiPoint"",""coordinates"":[]}"
9,S1A_IW_GRDH_1SDV_20220505T093153_20220505T0932...,_1SDV_20,S1A_IW_GRDH_1SDV_20220505T093153_20220505T0932...,"[-25.848216260662973, -26.567875803484114, -25...","[-26.346893167753183, -27.54052542050434, -25....",[],"[36.375929957493845, 36.375929957493845, 36.37...","[126.33232419393862, 126.33322250922274, 126.3...","{""type"":""MultiPoint"",""coordinates"":[]}"
